In [3]:
import numpy as np
import pandas as pd
import difflib #provides classes and functions for comparing sequences.
from sklearn.feature_extraction.text import TfidfVectorizer#converts a collection of raw text documents into a matrix of TF-IDF 
from sklearn.metrics.pairwise import cosine_similarity

## Data Colloection and pre-processing

In [4]:
df = pd.read_csv('movies.csv')

In [5]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
df.shape

(4803, 24)

In [7]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [8]:
#selecting features for recommendation
selected_F = ['genres', 'keywords', 'tagline', 'director']

In [9]:
#replacing null values with null string
for f in selected_F:
    df[f] = df[f].fillna('')

In [10]:
#combine all features
combined_F = df['genres']+''+df['keywords']+''+df['tagline']+''+df['director']

In [11]:
combined_F

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801                  A New Yorker in ShanghaiDaniel Hsia
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

## vectorizing:

In [12]:
#converting text data into featur vectors
vectorizer = TfidfVectorizer()

In [13]:
features_vec = vectorizer.fit_transform(combined_F)

In [14]:
print(features_vec)

  (0, 2038)	0.2648632646279761
  (0, 8249)	0.20712656467835527
  (0, 10837)	0.330993062181287
  (0, 10606)	0.1101724296171047
  (0, 16136)	0.15990910107709552
  (0, 14297)	0.09584668344399845
  (0, 13370)	0.3470108059059786
  (0, 2631)	0.310813091658875
  (0, 15651)	0.16548235704111564
  (0, 13458)	0.42662353084982124
  (0, 6599)	0.20948575016128987
  (0, 2506)	0.2796653857685013
  (0, 5933)	0.3470108059059786
  (0, 12832)	0.1262886886467352
  (0, 5665)	0.14959784992436587
  (0, 362)	0.11542645761684521
  (0, 202)	0.09957765762850049
  (1, 15463)	0.254480593141629
  (1, 6895)	0.22658486136172992
  (1, 1410)	0.20179831146391786
  (1, 5115)	0.20788466942633543
  (1, 8957)	0.30216866159987815
  (1, 10650)	0.1326727611338904
  (1, 9121)	0.13085224799157996
  (1, 3044)	0.23777928452683444
  :	:
  (4799, 14297)	0.09038337395618214
  (4800, 14166)	0.4130880444232522
  (4800, 9929)	0.4130880444232522
  (4800, 13318)	0.2857365072648403
  (4800, 10300)	0.3329188178978095
  (4800, 13179)	0.312447

## Cosine Similarity

In [15]:
#getting the similarity scores
similarity = cosine_similarity(features_vec)
similarity

array([[1.        , 0.11009452, 0.02551749, ..., 0.        , 0.        ,
        0.        ],
       [0.11009452, 1.        , 0.04767433, ..., 0.02340753, 0.        ,
        0.        ],
       [0.02551749, 0.04767433, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.02340753, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [16]:
similarity.shape

(4803, 4803)

## MOVIE RECOMMENDATION SYSTEM

In [17]:
#getting a movie name from my user

movie_user = input('You watched it, you want to watch something like it? Cool, just give us the name of it:')

You watched it, you want to watch something like it? Cool, just give us the name of it:iron man


In [18]:
#getting all the movie names from my dataset

list_of_names = df['title'].tolist()
list_of_names

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [20]:
#getting the close match of my user's movie
close_matchs = difflib.get_close_matches(movie_user, list_of_names)

In [21]:
close_matchs

['Iron Man', 'Iron Man 3', 'Iron Man 2']

In [22]:
close_match = close_matchs[0]

In [23]:
#finding the Index of close_match
index_movie = df[df.title == close_match]['index'].values[0]
index_movie

68

In [24]:
#getting a list of similar movies
similair = list(enumerate(similarity[index_movie]))
print(similair)

[(0, 0.023699584378131425), (1, 0.05314129726993078), (2, 0.009976555292613324), (3, 0.009909877755822449), (4, 0.023572760208165518), (5, 0.009864825464923776), (6, 0.03420488673877585), (7, 0.11276754479911391), (8, 0.0), (9, 0.05852089884366468), (10, 0.029629715294015015), (11, 0.00841573627159765), (12, 0.0), (13, 0.009444631768828534), (14, 0.06718928186669451), (15, 0.0), (16, 0.17588902613086932), (17, 0.009169733609166406), (18, 0.021775958045274022), (19, 0.007394819022262993), (20, 0.10564939075952491), (21, 0.007382735342115984), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.13842862656844254), (27, 0.021806512367743516), (28, 0.04543429219165713), (29, 0.010032581936694544), (30, 0.11226397393692193), (31, 0.174278251138283), (32, 0.030410681301575468), (33, 0.1975266476649884), (34, 0.0), (35, 0.05477580479789114), (36, 0.0484631807504225), (37, 0.0), (38, 0.16246183852005128), (39, 0.024663258472752574), (40, 0.0), (41, 0.021044520848748034), (42, 0.0), (43, 0.05051

In [25]:
#sorting the movies based on their similarity
sorted_movies = sorted(similair, key = lambda x:x[1], reverse = True)
sorted_movies

[(68, 1.0),
 (79, 0.2599146243534828),
 (3623, 0.21638283451299725),
 (33, 0.1975266476649884),
 (4401, 0.19136923004333134),
 (511, 0.18367096189079934),
 (46, 0.17706568252916713),
 (16, 0.17588902613086932),
 (31, 0.174278251138283),
 (203, 0.16403812423267924),
 (38, 0.16246183852005128),
 (174, 0.16092236979269967),
 (64, 0.15805385239691985),
 (788, 0.148150948311221),
 (101, 0.14787496933994956),
 (26, 0.13842862656844254),
 (182, 0.1323525913583304),
 (85, 0.13174213760690162),
 (2823, 0.13017799340077002),
 (4033, 0.1298025812664974),
 (661, 0.12694648289328228),
 (307, 0.12519893088803966),
 (4542, 0.12473641639141882),
 (3443, 0.11989137728877881),
 (1494, 0.11961860684390915),
 (2066, 0.11894135179831858),
 (1740, 0.11612337790035188),
 (7, 0.11276754479911391),
 (30, 0.11226397393692193),
 (1029, 0.1105940569556504),
 (3853, 0.10582224754097869),
 (20, 0.10564939075952491),
 (2395, 0.10459575251500608),
 (2221, 0.10403609924786128),
 (78, 0.10218685158632565),
 (129, 0.102

In [26]:
#printig the name of movies
print('Movies suggested for you : \n')

i = 1

for movie in sorted_movies:
    index = movie[0]
    title_from_index = df[df.index==index]['title'].values[0]
    if (i<10):
        print(i, '.',title_from_index)
        i+=1

Movies suggested for you : 

1 . Iron Man
2 . Iron Man 2
3 . Made
4 . X-Men: The Last Stand
5 . The Helix... Loaded
6 . X-Men
7 . X-Men: Days of Future Past
8 . The Avengers
9 . Iron Man 3


## Resume

1. Ask the user for the movie name, that we are searching for its similairs.
2. Find the most similair movie to it in our dataset **df** using **difflib.get_close_matchs(movie, list_of_names)**
3. Extracting the index of the movie, and find similair movies using **similarity[movie_index]**
4. Sorting the similair movies, using the function **sorted()**
5. printing the top simlair 10 movies